In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

course_code = "END"
course_crn = "12721"
student_level = "LS"

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Content-Type': 'application/x-www-form-urlencoded',
    'Origin': 'https://www.sis.itu.edu.tr',
    'Accept-Language': 'tr-tr',
    'Host': 'www.sis.itu.edu.tr',
    'Referer': f'https://www.sis.itu.edu.tr/TR/ogrenci/ders-programi/ders-programi.php?seviye={student_level}',
    'Connection': 'keep-alive'
}

params = {
    'seviye': f'{student_level}'
}


data = f'seviye={student_level}&derskodu={course_code}&B1=G%F6ster'

response = requests.post('https://www.sis.itu.edu.tr/TR/ogrenci/ders-programi/ders-programi.php', params=params, headers=headers, data=data)

soup = BeautifulSoup(response.text, 'lxml')
div_objects = soup.find_all("div", {"class":"content-area"})
texts = [r.text.strip() for r in div_objects][0]
last_update = texts[texts.find("Son Güncelleme: ")+16:]
table = soup.find_all('table')[0]
rows = table.find_all('tr')
course_table = pd.DataFrame(columns = range(0,15),index = range(0,len(rows)-2))
row_marker = 0
for row in rows:
    if row.attrs:
        if row.attrs['class'] == ['table-baslik']:
            a = row.find_all('td')
            new_list = [i.get_text().strip() for i in a]
            course_table.columns = list(new_list)
            continue
    column_marker = 0
    columns = row.find_all('td')
    for column in columns:
        course_table.iat[row_marker,column_marker] = column.get_text()
        column_marker += 1
    row_marker += 1
indus_40 = course_table.loc[course_table["CRN"] == course_crn]
if indus_40.empty:
    print(f"Course not found with given parameters: '{course_code}', '{course_crn}'")
    quit()
capacity = int(indus_40["Capacity"].values[0])
enrolled = int(indus_40["Enrolled"].values[0])
course_title = str(indus_40["Course Title"].values[0])
if capacity > enrolled:
    print(f"There is a spot in the class '{course_title} {course_crn}' ({enrolled}/{capacity})")
else:
    print(f"There is no spot in the class '{course_title} {course_crn}' ({enrolled}/{capacity})")
print(f"Last update: {last_update}")